In [11]:
!pip install geopandas pandas openpyxl

In [22]:
!pip install fuzzywuzzy

In [23]:
from fuzzywuzzy import process

/home/yuna/github/CyclingVSPollution/yuna/jupyter_env/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
import geopandas as gpd
import pandas as pd

In [13]:
# Define file paths
geojson_path = '/home/yuna/github/CyclingVSPollution/yuna/project_data/london_boroughs.geojson'  
excel_path = '/home/yuna/github/CyclingVSPollution/yuna/project_data/London_NO2_2023.xlsx'       

In [19]:
# Load GeoJSON data without geometry to avoid issues
geojson_data = gpd.read_file(geojson_path, driver="GeoJSON", ignore_geometry=True)

/home/yuna/github/CyclingVSPollution/yuna/jupyter_env/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [15]:
# Load Excel data
df = pd.read_excel(excel_path)

In [28]:
# Get unique borough names from GeoJSON
geojson_boroughs = geojson_data['name'].tolist()
geojson_boroughs

['Barking and Dagenham',
 'Barnet',
 'Bexley',
 'Brent',
 'Bromley',
 'Camden',
 'City of London',
 'Croydon',
 'Ealing',
 'Enfield',
 'Greenwich',
 'Hackney',
 'Hammersmith and Fulham',
 'Haringey',
 'Harrow',
 'Havering',
 'Hillingdon',
 'Hounslow',
 'Islington',
 'Kensington and Chelsea',
 'Kingston upon Thames',
 'Lambeth',
 'Lewisham',
 'Merton',
 'Newham',
 'Redbridge',
 'Richmond upon Thames',
 'Southwark',
 'Sutton',
 'Tower Hamlets',
 'Waltham Forest',
 'Wandsworth',
 'Westminster']

In [20]:
# Function to find the best match for each borough in the Excel data
def match_borough(borough_name, choices):
    best_match, score = process.extractOne(borough_name, choices)
    return best_match if score > 80 else None  # Use a threshold for accuracy

In [24]:
# Apply fuzzy matching to find borough names
df['borough_name'] = df['Borough'].apply(lambda x: match_borough(x, geojson_boroughs))

,Borough,Number of sites,Exceeding legal limit,Percent exceeding legal limit,Exceeding WHO,Percent exceeding WHO,Average concentration roadside*,Average concentration background**,Unnamed: 8,Unnamed: 9,borough_name
0,Corporation City of London,86,5.0,5.8,86.0,100.0,32.2,25.1,NaN,* includes Kerbside,City of London
1,LB Barking and Dagenham,50,0.0,0.0,50.0,100.0,26.6,17.9,NaN,**includes Urban Centre,Barking and Dagenham
2,LB Barnet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Barnet
3,LB Bexley,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bexley
4,LB Brent,127,2.0,1.6,127.0,100.0,22.5,19.0,NaN,NaN,Brent


In [38]:
df.loc[df['Borough'].str.contains('Waltham', case=False, na=False), 'borough_name'] = 'Waltham Forest'
df[df['Borough'] == 'LB Waltham']

,Borough,Number of sites,Exceeding legal limit,Percent exceeding legal limit,Exceeding WHO,Percent exceeding WHO,Average concentration roadside*,Average concentration background**,Unnamed: 8,Unnamed: 9,borough_name
27,LB Waltham,55,0.0,0.0,55.0,100.0,24.1,NaN,NaN,NaN,Waltham Forest


In [39]:
csv_output_path = 'london_with_borough_name.csv'
df.to_csv(csv_output_path, index=False)

In [25]:
# Inspect columns to find common keys
print("GeoJSON Columns:", geojson_data.columns)
print("Excel Columns:", df.columns)

GeoJSON Columns: Index(['name', 'color'], dtype='object')
Excel Columns: Index(['Borough', 'Number of sites', 'Exceeding legal limit',
       'Percent exceeding legal limit', 'Exceeding WHO',
       'Percent exceeding WHO', 'Average concentration roadside*',
       'Average concentration background**', 'Unnamed: 8', 'Unnamed: 9',
       'borough_name'],
      dtype='object')


In [26]:
# Merge the data using the common key (name and Borough)
merged_df = geojson_data.merge(df, left_on='name', right_on='Borough', how='inner')
merged_df

,name,color,Borough,Number of sites,Exceeding legal limit,Percent exceeding legal limit,Exceeding WHO,Percent exceeding WHO,Average concentration roadside*,Average concentration background**,Unnamed: 8,Unnamed: 9,borough_name


In [7]:
# Save the merged result as CSV
csv_output_path = 'merged_data.csv'
merged_df.to_csv(csv_output_path, index=False)

In [8]:
print(f"Merged data has been saved to {csv_output_path}")

Merged data has been saved to merged_data.csv
